In [ ]:
#Import all necessary libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm

In [ ]:
#All car brands that are met in the provided dataset
car_brand = ['bmw', 'volkswagen', 'nissan', 'mercedes','toyota',
         'audi', 'mitsubishi', 'skoda', 'volvo', 'honda',
         'infiniti', 'lexus']

In [ ]:
#Script for scraping
for brand in car_brand:
    #Create a dictionary with all the functions to derive from a website
    car_list = {'car_url': [], 'prod_year': [], 'mileage': [], 'body': [], 'color': [],
               'motor': [], 'transmission': [], 'gear': [], 'stir': [], 'condition':[],
               'owners': [], 'PTS': [], 'customs': [], 'price': []}
    
    pages_list = list(range(1, 11)) #Number of pages for scraping
    
    with tqdm(total=len(pages_list)) as pbar:
        err=0 #Error counter
        k=0   #Advertisement counter
        
        for i in pages_list:
            url = f'https://auto.ru/cars/{brand}/used/?output_type=list&page={i}'
            response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
            response.encoding = 'utf-8'
            soup = BeautifulSoup(response.text, 'html.parser')
            page = soup.find_all('a', class_ = 'Link ListingItemTitle-module__link')
            links_list=[]  #Create a list of links to car advertisements
            
            for j in range(len(page)):
                links_list.append(str(page[j]).split('"')[3])
                
            for link in links_list:
                k+=1
                car_soup = BeautifulSoup(requests.get(link, headers={'User-Agent': 'Mozila/5.0'}).text, 
                                         'html.parser')
                try:
                    car_soup.find_all(
                        'span', class_ = 'CardInfoRow__cell')[1].text.encode('l1').decode().replace('\xa0', '')
                    car_soup.find_all('span', class_ = 'OfferPriceCaption__price')[0]
                except:
                    err+=1
                    print(k, link, err)
                    print('Error')
                    continue # continue if a car has been already sold and no price is mentioned on the page
                car_list['car_url'].append(link)
                l=1
                
                for key in list(car_list.keys())[1:-1]:
                    car_list[key].append(car_soup.find_all(
                        'span', class_ = 'CardInfoRow__cell')[l].text.encode('l1').decode().replace('\xa0', ''))
                    l+=2
                    
                car_list['price'].append(car_soup.find_all(
                    'span', class_ = 'OfferPriceCaption__price')[0].text.encode('l1').decode().replace('\xa0', ''))
            pbar.update(1)
            
    a = pd.DataFrame.from_dict(car_list, orient='columns') # create a dataframe from the obtained dictionary
    a.to_csv(f'{brand}.csv',index=False) # save the dataframe as csv file

In [ ]:
#Combine all dataframes in one dataset
data = pd.read_csv(f'bmw.csv')
for brand in car_brand[1::]:
    brand = pd.read_csv(f'{brand}.csv')
    data = pd.concat([data, brand])

In [ ]:
#Drop duplicates
data = data.drop_duplicates(data.drop_duplicates(subset=['prod_year',
 'mileage',
 'body',
 'color',
 'motor',
 'transmission',
 'gear',
 'stir',
 'condition',
 'owners',
 'PTS',
 'customs',
 'price'], inplace=True))
data.info()

In [ ]:
#The obtained dataset does not contain any cars with body type = limousine.
#Address this target.

car_list = {'car_url': [], 'prod_year': [], 'mileage': [], 'body': [], 'color': [],
            'motor': [], 'transmission': [], 'gear': [], 'stir': [], 'condition':[],
            'owners': [], 'PTS': [], 'customs': [], 'price': []}
pages_list = list(range(1, 2))

with tqdm(total=len(pages_list)) as pbar:
    err=0
    k=0
    url = f'https://auto.ru/cars/mercedes/used/?body_type_group=LIMOUSINE'
    response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
    response.encoding = 'utf-8'
    soup = BeautifulSoup(response.text, 'html.parser')
    page = soup.find_all('a', class_ = 'Link ListingItemTitle-module__link')
    links_list=[]
    
    for j in range(len(page)):
        links_list.append(str(page[j]).split('"')[3])
        
    for link in links_list:
        k+=1
        car_soup = BeautifulSoup(requests.get(link, headers={'User-Agent': 'Mozila/5.0'}).text, 'html.parser')
        
        try:
            car_soup.find_all(
                'span', class_ = 'CardInfoRow__cell')[1].text.encode('l1').decode().replace('\xa0', '')
            car_soup.find_all('span', class_ = 'OfferPriceCaption__price')[0]
        except:
            err+=1
            prnit(k, link, err)
            print('Error')
            continue
        car_list['car_url'].append(link)
        l=1
        
        for key in list(car_list.keys())[1:-1]:
            car_list[key].append(car_soup.find_all(
                'span', class_ = 'CardInfoRow__cell')[l].text.encode('l1').decode().replace('\xa0', ''))
            l+=2
            
        car_list['price'].append(car_soup.find_all(
            'span', class_ = 'OfferPriceCaption__price')[0].text.encode('l1').decode().replace('\xa0', ''))
    pbar.update(1)
    
a = pd.DataFrame.from_dict(car_list, orient='columns')
a.to_csv(f'mercedes_limousine.csv',index=False)

